### 当前实验模型内容

1. use mentions
2. 63.3

3. add passage score

model_name |  param | dev_acc|
---| --- | ---
use mentions | lr=1e-3,hidden=50 | 63.3

In [1]:
import os
import torch
import torch.nn as nn
import torchtext
from tensorboardX import SummaryWriter
import random
import numpy as np

from torchtext.data import NestedField, Field, RawField
from model import *
from dataset import DataHandler
%load_ext autoreload

%autoreload 2
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

03/05/2019 21:03:42 - INFO - summarizer.preprocessing.cleaner -   'pattern' package not found; tag filters are not available for English


In [2]:
class Config:
    def __init__(self):
        self.hidden = 50
        self.embedding_dim = 300 + 100
        self.lr = 1e-4
        self.epochs = 30
        self.fix_length = None
        
        self.log_dir = './logs'
        self.model_name = 'CFC_um_ps'
        self.batch_size = 4
        self.train_data = './data/train_filter.pt'
        self.dev_data = './data/dev_filter.pt'
        
        self.word_vocab = './data/glove_vocab.pt'
        self.charNGram_vocab = './data/charNGram_vocab.pt'
        
        self.dropout = 0.2
        self.seed = 1023
        
config = Config()
device = torch.device("cuda:0")


In [3]:
random.seed(config.seed)
np.random.seed(config.seed)
torch.manual_seed(config.seed)
torch.cuda.manual_seed_all(config.seed)

In [4]:
save_path = config.model_name  + '_lr_'+ str(config.lr)+ '__hidden__' + str(config.hidden) \
            + '_batchsize_' + str(config.batch_size) +  '_p'+ str(config.dropout)
save_path = os.path.join(config.log_dir, save_path)   
print(save_path)
config.save_path = save_path

./logs/CFC_um_ps_lr_0.0001__hidden__50_batchsize_4_p0.2


### Define Fileds

In [5]:
word_field = Field(batch_first=True, sequential=True, tokenize="spacy", lower=True) # query
multi_word_field = NestedField(word_field) 

word_field_sup = Field(batch_first=True, sequential=True, tokenize="spacy", lower=True, fix_length=config.fix_length)
multi_word_field_sup = NestedField(word_field_sup) 

charNGram_field = Field(batch_first=True, sequential=True, tokenize="spacy", lower=True) # query
multi_charNGram_field = NestedField(charNGram_field) 

charNGram_field_sup = Field(batch_first=True, sequential=True, tokenize="spacy", lower=True, fix_length=config.fix_length)
multi_charNGram_field_sup = NestedField(charNGram_field_sup) 

raw = RawField()
raw.is_target = False

label_field = Field(sequential=False, is_target=True, use_vocab=False)

dict_field = {
    'id': ('id', raw),
    'supports': [('s_glove', multi_word_field_sup), ('s_charNGram', multi_charNGram_field_sup)],
    'query': [('q_glove', word_field), ('q_charNGram', charNGram_field)],
    'candidates': [('c_glove', multi_word_field), ('c_charNGram', multi_charNGram_field)],
    'label': ('label', label_field),
    'mentions': ('mentions', raw),
    'para_label': ('para_label', raw)
}

In [6]:
data_handler = DataHandler(config.train_data, config.dev_data, dict_field)

# torch.save(data_handler.trainset.examples, './data/train_example.pt')
# torch.save(data_handler.valset.examples, './data/dev_example.pt')

load examples.pt  :./data/train_filter.pt, ./data/dev_filter.pt


In [7]:
from tqdm import tqdm

def add_mentions(examples):

    for example in tqdm(examples):
        candidates = example.c_glove
        supports = example.s_glove


        all_mentions = []

        for candidate in candidates:
            mentions = []
            c = ' '.join(candidate)
            for idx, support in enumerate(supports):

                for i in range(len(support)):
                    token = support[i]
                    if token == candidate[0]:
                        s = ' '.join(support[i:i+len(candidate)])
                        if s == c:
                            mentions.append([idx, i, i+len(candidate)])
            all_mentions.append(mentions)
            
        example.mentions = all_mentions
        
def add_para_label(examples):
    filter_examples=  []
    for example in tqdm(examples):
        candidates = example.c_glove
        supports = example.s_glove    

        label = example.label
        mentions = example.mentions
        answser_mentions = mentions[label]
        if len(answser_mentions) != 0:
            para_label = [0]*len(supports)
            for mentions in answser_mentions:
                para_label[mentions[0]] = 1
            example.para_label = para_label
            filter_examples.append(example)
    print(f'before filter: {len(examples)}, after:{len(filter_examples)}')
    return filter_examples

#add_mentions(data_handler.valset.examples)
#add_mentions(data_handler.trainset.examples)

#train_filter_examples  = add_para_label(data_handler.trainset.examples)
#dev_filter_examples  = add_para_label(data_handler.valset.examples)

#torch.save(train_filter_examples, './data/train_filter.pt')
#torch.save(dev_filter_examples, './data/dev_filter.pt')        

In [8]:
#add_mentions(data_handler.valset.examples)
#add_mentions(data_handler.trainset.examples)

### Build Vocab

In [9]:
if config.charNGram_vocab is not None:
    charNGram_vocab = torch.load(config.charNGram_vocab)
    charNGram_field_sup.vocab = charNGram_vocab
else:
    charNGram_field_sup.build_vocab(data_handler.trainset, data_handler.valset, 
                                          vectors=torchtext.vocab.CharNGram())

if config.word_vocab is not None:
    word_vocab = torch.load(config.word_vocab)
    word_field_sup.vocab = word_vocab
else:
    word_field_sup.build_vocab(data_handler.trainset, data_handler.valset, 
                                 vectors=torchtext.vocab.GloVe(dim=300,name='6B') )

word_field.vocab = word_field_sup.vocab
charNGram_field.vocab = charNGram_field_sup.vocab

# torch.save(word_field.vocab, './data/glove_vocab.pt')
# torch.save(charNGram_field.vocab, './data/charNGram_vocab.pt')

### Get data_iter

In [10]:
train_iter = data_handler.get_train_iter(batch_size=config.batch_size)
val_iter = data_handler.get_val_iter(batch_size=config.batch_size)

In [11]:
for idx, batch in enumerate(val_iter):
    break
batch


[torchtext.data.batch.Batch of size 4]
	[.id]:['WH_dev_0', 'WH_dev_1', 'WH_dev_2', 'WH_dev_3']
	[.s_glove]:[torch.LongTensor of size 4x15x292]
	[.s_charNGram]:[torch.LongTensor of size 4x15x292]
	[.q_glove]:[torch.LongTensor of size 4x11]
	[.q_charNGram]:[torch.LongTensor of size 4x11]
	[.c_glove]:[torch.LongTensor of size 4x18x4]
	[.c_charNGram]:[torch.LongTensor of size 4x18x4]
	[.label]:[torch.LongTensor of size 4]
	[.mentions]:[[[[6, 145, 146], [6, 173, 174], [7, 78, 79]], [[3, 50, 53], [5, 28, 31], [13, 1, 4]], [[6, 135, 136], [6, 218, 219], [6, 261, 262], [8, 45, 46], [12, 98, 99]], [[0, 2, 4], [7, 1, 3], [13, 64, 66], [13, 69, 71]], [[0, 36, 38], [10, 1, 3], [13, 75, 77]], [[0, 14, 15], [1, 63, 64], [1, 138, 139], [1, 186, 187], [1, 238, 239], [2, 128, 129], [9, 8, 9], [9, 43, 44], [10, 19, 20], [10, 34, 35], [11, 37, 38], [11, 79, 80], [13, 56, 57]], [[7, 37, 40]], [[6, 180, 181], [12, 101, 102]], [[12, 96, 97]], [[8, 43, 46]], [[6, 169, 172]], [[6, 179, 181]], [[7, 38, 40]], 

In [12]:
def get_para_label(batch):
    para_size = batch.s_glove.size(1)
    results = []
    for label in batch.para_label:
        padding = [0]*(para_size - len(label))
        n_label = label[:]
        n_label += padding
        results.append(n_label)
    results = torch.tensor(results, dtype=torch.long)
    return results

### Define Model

In [13]:
class SimpleQANet(nn.Module):
    
    def __init__(self, config, word_vectors, charNGram_vectors, device):
        super(SimpleQANet, self).__init__()
        self.config = config
        self.device = device
        
        self.embedding_layer = EmbeddingLayer(word_vectors, charNGram_vectors)
        

        self.rnn = EncoderRNN(config.embedding_dim, config.hidden, 1, True, True, config.dropout, False)
                
            
        self.co_att = CoAttention(config.hidden*2, att_type=2, dropout=config.dropout)
        
        self.linear_1 = nn.Sequential(
                        nn.Linear(config.hidden*4, config.hidden),
                        nn.ReLU()
                    )        
        self.rnn2 =  EncoderRNN(config.hidden, config.hidden, 1, True, True, config.dropout, False)
        
        self.word_att = SelfAttention(config.hidden*2, config.hidden*2, config.dropout)
        self.word_att_q = SelfAttention(config.hidden*2, config.hidden*2, config.dropout)
        self.p_score = BilinearSeqAttn(config.hidden*2, config.hidden*2, identity=False, dropout=config.dropout)
        
        self.pass_att = SelfAttention(config.hidden*2, config.hidden*2, config.dropout)
        
        self.c_att = SelfAttention(config.hidden*2, config.hidden*2, config.dropout)
        
        self.mention_att = SelfAttention(config.hidden*2, config.hidden*2, config.dropout)
        
        
        #self.fusion = FusionLayer(config.hidden*2, dropout=config.dropout)
        self.max_pooling = PoolingLayer()     
        
        self.fc = nn.Linear(config.hidden*2, config.hidden*4)
        
        self.to(device)
        
    def forward(self, batch, return_label = True):
        if type(batch.q_glove) is tuple:
            q_glove, _ = batch.q_glove
            q_charNGram, _ = batch.q_charNGram
        else:
            q_glove = batch.q_glove
            q_charNGram = batch.q_charNGram            
        
        s_glove = batch.s_glove
        s_charNGram = batch.s_charNGram
        
        c_glove = batch.c_glove
        c_charNGram = batch.c_charNGram
        
        q_glove = q_glove.to(self.device)
        q_charNGram = q_charNGram.to(self.device)

        s_glove = s_glove.to(self.device)
        s_charNGram = s_charNGram.to(self.device)

        c_glove = c_glove.to(self.device)
        c_charNGram = c_charNGram.to(self.device)
        
        ### Embedding and Encoder
        
        q_out = self.embedding_layer(q_glove, q_charNGram)
        s_out = self.embedding_layer(s_glove, s_charNGram,)
        c_out = self.embedding_layer(c_glove, c_charNGram)
        
        batch_size=  s_out.size(0)
        
        s_len = s_out.size(1)
        c_len = c_out.size(1)
        
        s_word_len = s_out.size(2)
        c_word_len = c_out.size(2)
        
        hidden = s_out.size(-1)
        
        s_out = s_out.view(batch_size*s_len, s_word_len, hidden).contiguous()
        c_out = c_out.view(batch_size*c_len, c_word_len, hidden).contiguous()
        
        q_out = self.rnn(q_out)
        c_out = self.rnn(c_out)
        s_out = self.rnn(s_out)
        
        # Attention
        
        q_word_len = q_out.size(1)
        q_out_expand = q_out.unsqueeze(1).expand(batch_size, s_len, q_word_len, q_out.size(-1)).contiguous()
        q_out_expand = q_out_expand.view(batch_size*s_len, q_word_len, q_out.size(-1)).contiguous()
        
        s_out_att, q_out_att = self.co_att(s_out, q_out_expand)
        #S_s = self.fusion(s_out, s_out_att)
        #S_q = self.fusion(q_out, q_out_att)
        
        S_s = self.linear_1(s_out_att)
        S_s = self.rnn2(S_s)
        
        
        
        batch_c_m = []
        for i in range(batch_size):
            # get mention embedding
            mentions = batch.mentions[i]
            c_ms = torch.zeros(c_len, s_len, s_out.size(-1))
            for idx, c_mention in enumerate(mentions):
                c_m_dict = {}
                for mention in c_mention:
                    m = s_out[i*s_len + mention[0]][mention[1]:mention[2]]
                    m = self.max_pooling(m.unsqueeze(0)).squeeze()
                    if mention[0] not in c_m_dict:
                        c_m_dict[mention[0]] = []
                    c_m_dict[mention[0]].append(m)
                c_m = torch.zeros(s_len, s_out.size(-1))
                for key in c_m_dict:
                    for m in c_m_dict[key]:
                        c_m[key] += m.cpu()
                    c_m[key] /= len(c_m_dict[key])
                c_ms[idx] = c_m
            batch_c_m.append(c_ms)
        batch_c_m = torch.stack(batch_c_m)
        batch_c_m = batch_c_m.to(self.device)
        batch_c_m = batch_c_m.view(batch_size*c_len, s_len, -1)
        batch_c_m = self.mention_att(batch_c_m)
        batch_c_m = batch_c_m.view(batch_size, c_len, -1)
        

        
        C_s = self.word_att(S_s)
        C_q = self.word_att_q(q_out)
        
        C_s = C_s.view(batch_size, s_len, -1)
        
        P_scores = self.p_score(C_s, C_q)
        
        C_s = self.pass_att(C_s) 

        
        C_c = self.c_att(c_out)        
        C_c = C_c.view(batch_size, c_len, -1)
        
        C_c = torch.cat([C_c, batch_c_m],-1)
        
        C_s = torch.tanh(self.fc(C_s))
        
        score = torch.bmm(C_c, C_s.unsqueeze(-1))
        score = score.squeeze(-1)
        if return_label:
            label = batch.label.to(self.device)
            P_label = get_para_label(batch)
            P_label = P_label.to(device)
            return score, P_scores, label, P_label
        return score, P_scores

#### test model

In [14]:
model = SimpleQANet(config, word_field.vocab.vectors, charNGram_field.vocab.vectors, device)
#score,P_score, label, P_label = model(batch)
#print(score, label, P_score, P_label)
#print(score.shape, label.shape, P_score.shape, P_label.shape)

In [15]:
from utils import AverageMeter

def train(epoch, data_iter, model, criterion, criterion_bce, optimizer, batch_size=1, joint_begin=-1):
    losses = AverageMeter()
    acces = AverageMeter()
    model.train()
    #model.embedding_layer.eval()
    for idx, batch in enumerate(data_iter):
        score, P_score, label, P_label = model(batch)

        loss1 = criterion(score, label)
        loss_p = criterion_bce(P_score, P_label.float())
        if epoch > joint_begin:
            loss = loss1 + loss_p     
        else:
            loss = loss_p

        loss = loss / batch_size
        loss.backward()
        if (idx+1)%batch_size == 0 :
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.25)            
            optimizer.step()
            optimizer.zero_grad()        

        losses.update(loss.item()*batch_size)
        
        pred = score.argmax(1)
        acc = pred.eq(label).sum().item()  / pred.size(0)
        acces.update(acc)
        if (idx+1) % (batch_size*100) == 0:
            print(f'epoch:{epoch}, idx:{idx}/{len(data_iter)}, loss:{losses.avg}, acc:{acces.avg}')
    return losses.avg, acces.avg

def val(epoch, data_iter, model, criterion, criterion_bce, joint_begin=-1):
    losses = AverageMeter()
    acces = AverageMeter()
    model.eval()
    for idx, batch in enumerate(data_iter):
        with torch.no_grad():
            score, P_score, label, P_label = model(batch)
                    
        loss1 = criterion(score, label)
        loss_p = criterion_bce(P_score, P_label.float())
        if epoch > joint_begin:
            loss = loss1 + loss_p     
        else:
            loss = loss_p
        
        losses.update(loss.item())
        
        pred = score.argmax(1)
        acc = pred.eq(label).sumbest
            print(f'epoch:{epoch}, idx:{idx}/{len(data_iter)}, loss:{losses.avg}, acc:{acces.avg}')
    return losses.avg, acces.avg

In [16]:
optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, model.parameters()),
                             lr=config.lr)

criterion = nn.CrossEntropyLoss()
criterion_bce = nn.BCEWithLogitsLoss()

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config.epochs)
#train(0, train_iter, model, criterion, optimizer, batch_size=config.batch_size)
# val(0, val_iter, model,criterion)

In [17]:
if not os.path.exists(config.save_path):
    os.makedirs(config.save_path)
writer = SummaryWriter(config.save_path)

best_acc = 0.0
for epoch in range(config.epochs):
    scheduler.step()
    train_loss, train_acc = train(epoch, train_iter, model, criterion, criterion_bce, optimizer, 
                                     1)
    val_loss, val_acc = val(epoch, val_iter, model, criterion, criterion_bce)
    writer.add_scalar('train_loss', train_loss, epoch+1)
    writer.add_scalar('val_loss', val_loss, epoch+1)
    writer.add_scalar('train_acc', train_acc, epoch+1)
    writer.add_scalar('val_acc', val_acc, epoch+1)
    
    state = {
        'val_acc': val_acc,
        'train_acc': train_acc,
        'epoch': epoch,
        'model': model.state_dict()
    }
    #torch.save(state, os.path.join(config.save_path,'lastest.pth'))
    if val_acc > best_acc:
        best_acc = val_acc
        #torch.save(state, os.path.join(save_path, 'best.pth'))

/home/user2/Multi_QA/model.py:184: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  alphas = self.softmax(alphas)  # (bsz, sent_len)


epoch:0, idx:99/10845, loss:3.8411771512031554, acc:0.1875
epoch:0, idx:199/10845, loss:3.464788509607315, acc:0.22375
epoch:0, idx:299/10845, loss:3.2804538949330646, acc:0.24666666666666667
epoch:0, idx:399/10845, loss:3.129928788244724, acc:0.26375
epoch:0, idx:499/10845, loss:3.036080674648285, acc:0.2775
epoch:0, idx:599/10845, loss:2.951693090200424, acc:0.28708333333333336
epoch:0, idx:699/10845, loss:2.903557346037456, acc:0.2932142857142857
epoch:0, idx:799/10845, loss:2.8517998991906643, acc:0.300625
epoch:0, idx:899/10845, loss:2.799626319143507, acc:0.3080555555555556
epoch:0, idx:999/10845, loss:2.7602129591703415, acc:0.315
epoch:0, idx:1099/10845, loss:2.739209978688847, acc:0.31727272727272726
epoch:0, idx:1199/10845, loss:2.7170045927166937, acc:0.31833333333333336
epoch:0, idx:1299/10845, loss:2.691451080579024, acc:0.3219230769230769
epoch:0, idx:1399/10845, loss:2.6678093465736934, acc:0.3267857142857143
epoch:0, idx:1499/10845, loss:2.644142016331355, acc:0.3303333

epoch:0, idx:900/1275, loss:1.9913261348875726, acc:0.43784683684794673
epoch:0, idx:1000/1275, loss:2.0048809521324507, acc:0.4373126873126873
epoch:0, idx:1100/1275, loss:2.002054981271534, acc:0.4391462306993642
epoch:0, idx:1200/1275, loss:2.0064881068284466, acc:0.4377601998334721
epoch:1, idx:99/10845, loss:1.9928994929790498, acc:0.42
epoch:1, idx:199/10845, loss:1.9036356392502785, acc:0.47
epoch:1, idx:299/10845, loss:1.925157831509908, acc:0.4666666666666667
epoch:1, idx:399/10845, loss:1.9418779096007348, acc:0.46125
epoch:1, idx:499/10845, loss:1.9272917999625205, acc:0.4665
epoch:1, idx:599/10845, loss:1.9131777634720009, acc:0.4675
epoch:1, idx:699/10845, loss:1.9142278436677798, acc:0.4660714285714286
epoch:1, idx:799/10845, loss:1.9173747937381267, acc:0.4653125
epoch:1, idx:899/10845, loss:1.9155907676617305, acc:0.4666666666666667
epoch:1, idx:999/10845, loss:1.9038710365891456, acc:0.47075
epoch:1, idx:1099/10845, loss:1.9090830592133783, acc:0.46863636363636363
epoc

epoch:1, idx:400/1275, loss:1.76406916418575, acc:0.506857855361596
epoch:1, idx:500/1275, loss:1.7602203544266448, acc:0.5054890219560878
epoch:1, idx:600/1275, loss:1.7573173126444444, acc:0.5045757071547421
epoch:1, idx:700/1275, loss:1.7476601326006456, acc:0.507132667617689
epoch:1, idx:800/1275, loss:1.7648820746108684, acc:0.49812734082397003
epoch:1, idx:900/1275, loss:1.7603050737613843, acc:0.4997225305216426
epoch:1, idx:1000/1275, loss:1.7751759862030423, acc:0.4985014985014985
epoch:1, idx:1100/1275, loss:1.7698245346546173, acc:0.49909173478655766
epoch:1, idx:1200/1275, loss:1.773644283922586, acc:0.49646128226477937
epoch:2, idx:99/10845, loss:1.745439372062683, acc:0.4925
epoch:2, idx:199/10845, loss:1.7044200116395951, acc:0.51875
epoch:2, idx:299/10845, loss:1.6579422881205876, acc:0.5216666666666666
epoch:2, idx:399/10845, loss:1.6883791287243366, acc:0.518125
epoch:2, idx:499/10845, loss:1.7073645687103272, acc:0.5165
epoch:2, idx:599/10845, loss:1.704043060441812,

epoch:2, idx:0/1275, loss:1.5896270275115967, acc:0.5
epoch:2, idx:100/1275, loss:1.7734209139748376, acc:0.5024752475247525
epoch:2, idx:200/1275, loss:1.7134486166991998, acc:0.5211442786069652
epoch:2, idx:300/1275, loss:1.706498766658314, acc:0.5282392026578073
epoch:2, idx:400/1275, loss:1.6984842574209942, acc:0.529925187032419
epoch:2, idx:500/1275, loss:1.6933465927184936, acc:0.530439121756487
epoch:2, idx:600/1275, loss:1.694623831603769, acc:0.5262063227953411
epoch:2, idx:700/1275, loss:1.6859435581106603, acc:0.5278174037089871
epoch:2, idx:800/1275, loss:1.7028886073091056, acc:0.5205992509363296
epoch:2, idx:900/1275, loss:1.6948982665065127, acc:0.5208102108768036
epoch:2, idx:1000/1275, loss:1.7057320207923086, acc:0.5202297702297702
epoch:2, idx:1100/1275, loss:1.7023425898315905, acc:0.5197547683923706
epoch:2, idx:1200/1275, loss:1.7065847133170755, acc:0.5172772689425479
epoch:3, idx:99/10845, loss:1.6549401676654816, acc:0.55
epoch:3, idx:199/10845, loss:1.6347173

...

epoch:25, idx:10399/10845, loss:1.1657176706782326, acc:0.6934615384615385
epoch:25, idx:10499/10845, loss:1.1656196262460379, acc:0.6933095238095238
epoch:25, idx:10599/10845, loss:1.1663070571148453, acc:0.6928301886792453
epoch:25, idx:10699/10845, loss:1.1668392329663038, acc:0.6928971962616822
epoch:25, idx:10799/10845, loss:1.1668866398592515, acc:0.6929861111111111
epoch:25, idx:0/1275, loss:1.4720144271850586, acc:0.5
epoch:25, idx:100/1275, loss:1.4879711115419274, acc:0.5891089108910891
epoch:25, idx:200/1275, loss:1.4021311114676556, acc:0.6231343283582089
epoch:25, idx:300/1275, loss:1.4058146127236641, acc:0.6320598006644518
epoch:25, idx:400/1275, loss:1.3859416462313803, acc:0.6390274314214464
epoch:25, idx:500/1275, loss:1.3793396831094147, acc:0.6417165668662674
epoch:25, idx:600/1275, loss:1.3900528666183676, acc:0.6356073211314476
epoch:25, idx:700/1275, loss:1.3893456785978162, acc:0.6355206847360912
epoch:25, idx:800/1275, loss:1.4114160044734994, acc:0.62702871410

epoch:26, idx:9799/10845, loss:1.1577055938108538, acc:0.695076530612245
epoch:26, idx:9899/10845, loss:1.1578515647901129, acc:0.6952272727272727
epoch:26, idx:9999/10845, loss:1.1585161117486655, acc:0.695275
epoch:26, idx:10099/10845, loss:1.1578396062678336, acc:0.6956435643564356
epoch:26, idx:10199/10845, loss:1.1576449860493634, acc:0.6956617647058824
epoch:26, idx:10299/10845, loss:1.1576211756272512, acc:0.6955582524271845
epoch:26, idx:10399/10845, loss:1.156689567430518, acc:0.6956490384615385
epoch:26, idx:10499/10845, loss:1.1567295097942862, acc:0.6957619047619048
epoch:26, idx:10599/10845, loss:1.1571058403046908, acc:0.6954716981132075
epoch:26, idx:10699/10845, loss:1.1571640799111016, acc:0.695677570093458
epoch:26, idx:10799/10845, loss:1.1574124034832198, acc:0.6956712962962963
epoch:26, idx:0/1275, loss:1.4651556015014648, acc:0.5
epoch:26, idx:100/1275, loss:1.4890092564101267, acc:0.594059405940594
epoch:26, idx:200/1275, loss:1.4021500835371254, acc:0.6243781094

epoch:27, idx:9199/10845, loss:1.152163189880388, acc:0.6972826086956522
epoch:27, idx:9299/10845, loss:1.151842332226775, acc:0.6974193548387096
epoch:27, idx:9399/10845, loss:1.151303015627918, acc:0.6974468085106383
epoch:27, idx:9499/10845, loss:1.1516081302346368, acc:0.6976052631578947
epoch:27, idx:9599/10845, loss:1.1524071505176834, acc:0.69734375
epoch:27, idx:9699/10845, loss:1.152386342541305, acc:0.6976030927835052
epoch:27, idx:9799/10845, loss:1.1527861019535637, acc:0.6975765306122449
epoch:27, idx:9899/10845, loss:1.1526855787070411, acc:0.6976262626262626
epoch:27, idx:9999/10845, loss:1.1526515168763698, acc:0.697425
epoch:27, idx:10099/10845, loss:1.1532154527210658, acc:0.697029702970297
epoch:27, idx:10199/10845, loss:1.1541683128658755, acc:0.6966421568627451
epoch:27, idx:10299/10845, loss:1.1550441324645744, acc:0.6964563106796117
epoch:27, idx:10399/10845, loss:1.1548021717434032, acc:0.6965625
epoch:27, idx:10499/10845, loss:1.155598604562027, acc:0.696619047

epoch:28, idx:8599/10845, loss:1.144960397368427, acc:0.6980523255813953
epoch:28, idx:8699/10845, loss:1.1440716171478746, acc:0.6983045977011494
epoch:28, idx:8799/10845, loss:1.1443865636270494, acc:0.6984659090909091
epoch:28, idx:8899/10845, loss:1.14554932339389, acc:0.6982022471910112
epoch:28, idx:8999/10845, loss:1.1462508687153459, acc:0.6978888888888889
epoch:28, idx:9099/10845, loss:1.147279826360908, acc:0.6975274725274725
epoch:28, idx:9199/10845, loss:1.1477866055225225, acc:0.6972554347826087
epoch:28, idx:9299/10845, loss:1.1480240923238378, acc:0.6970967741935484
epoch:28, idx:9399/10845, loss:1.1488929893765996, acc:0.6971010638297872
epoch:28, idx:9499/10845, loss:1.1492168651748644, acc:0.6969473684210526
epoch:28, idx:9599/10845, loss:1.1479441863182, acc:0.697109375
epoch:28, idx:9699/10845, loss:1.1476779639528891, acc:0.6970618556701031
epoch:28, idx:9799/10845, loss:1.148281012450402, acc:0.6968112244897959
epoch:28, idx:9899/10845, loss:1.1495936377414249, ac

epoch:29, idx:8099/10845, loss:1.1546135198407703, acc:0.6957407407407408
epoch:29, idx:8199/10845, loss:1.1544966800983358, acc:0.6957621951219513
epoch:29, idx:8299/10845, loss:1.1534244306995927, acc:0.6961144578313253
epoch:29, idx:8399/10845, loss:1.1539023181317107, acc:0.6960416666666667
epoch:29, idx:8499/10845, loss:1.1542593085537938, acc:0.696
epoch:29, idx:8599/10845, loss:1.1544232179657665, acc:0.6960755813953489
epoch:29, idx:8699/10845, loss:1.154519503906198, acc:0.6961494252873563
epoch:29, idx:8799/10845, loss:1.1531330983950334, acc:0.6964204545454545
epoch:29, idx:8899/10845, loss:1.1537459639551935, acc:0.6964044943820225
epoch:29, idx:8999/10845, loss:1.1533500792698728, acc:0.6965
epoch:29, idx:9099/10845, loss:1.1541304354508828, acc:0.6964010989010989
epoch:29, idx:9199/10845, loss:1.1546784833494736, acc:0.696304347826087
epoch:29, idx:9299/10845, loss:1.1536994060697734, acc:0.6966666666666667
epoch:29, idx:9399/10845, loss:1.1550267982926774, acc:0.69630319

In [ ]:
score, P_score, label, P_label = model(batch)
        
loss1 = criterion(score, label)
loss_p = criterion_bce(P_score, P_label.float())

loss = loss1 + loss_p

loss.backward()
#optimizer.step()
#optimizer.zero_grad()        

In [ ]:
model.p_score.linear.weight.grad

In [ ]:
model.mention_att.ws1.weight.grad

In [ ]:
outputs = []
def hook(module, input, output):
    outputs.append(output)

In [ ]:
hook1 = model.fc.register_forward_hook()
hook12 = model.